In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from tobler.util import h3fy

from DataAnalysis import DataPreparation as DataPreparation
from DataAnalysis import DataMethodPreparation as DataMethodPreparation
from AssastmentMethod import AspectLaw as AspectLaw
from AssastmentMethod import AspectPhysical as AspectPhysical
from AssastmentMethod import AspectSpatial as AspectSpatial
from AssastmentMethod import AspectEconomic as AspectEconomic
from AssastmentMethod import General as General

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Way51\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Загрузка исходного файла с ОКН
heritage_origin = pd.read_csv('Example_files/All_heritage.csv')

# Выборка объектов в нужном регионе
REGION = 'Вологодская область'
ch_region, ch_region_coor, ch_region_none = DataPreparation.separator_reestr(heritage_origin, REGION) # Общий датасет, ГДФ(что можно найти), ДФ без координат оставшиеся

100%|██████████| 975/975 [00:00<00:00, 4336.05it/s]


0    Вологодская область
1    Вологодская область
2    Вологодская область
Name: Регион, dtype: object
Всего объектов: 975
Количество объектов с координатами: 292
Количество объектов без координат: 683


In [4]:
# Загрузка отчета о кадастровой оценке
report = gpd.read_file('Example_files/buildings_report.geojson') 
report = DataPreparation.number_report(report)

h:\Documents\Python\Risks_of_investing_in_cultural_heritage\DataAnalysis.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35-164339' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[i, 'учетный номер'] = desc


In [5]:
# Выборка необходимых столбцов для анализа
ch_region = ch_region[['Номер в реестре', 'учетный номер', 'Объект', 'Регион', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'описание предмета охраны', 'Принадлежность к Юнеско', 'наименование документа', 'building_year', 'lat', 'lon', 'geometry']]
report = report[['Номер в реестре','учетный номер', 'Кадастровый номер', 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН', 'Кадастровые номера помещений в объекте недвижимости', 'Площадь, кв.м', 'Назначение здания', 'Год завершения строительства', 'Количество этажей (в том числе подземных)', 'Количество подземных этажей', 'Материал наружных стен', 'Класс конструктивной системы', 'Группа капитальности', 'Объект признан аварийным ', 'Функциональная группа', 'Подгруппа', 'usage', 'area', 'cad_cost', 'geometry']]

# Преобразование столбцов в нужный формат
ch_region['Номер в реестре'] = ch_region['Номер в реестре'].astype(str)
ch_region['учетный номер'] = ch_region['учетный номер'].astype(str)

In [6]:
# Разделение в зависимости от наличие информации
report_1 = report[report['Номер в реестре'] != 0]
report_2 = report[report['Номер в реестре'] == 0]

# объединяем и удалаяем лишние стобцы
gdf_merged_1 = pd.merge(report_1, ch_region, on='Номер в реестре', how='inner')
gdf_merged_2 = pd.merge(report_2, ch_region, on='учетный номер', how='inner')

gdf_merged = pd.concat([gdf_merged_1, gdf_merged_2], ignore_index=True)
gdf_merged = gdf_merged.drop(['Номер в реестре_x', 'Номер в реестре_y', 'учетный номер_y', 'учетный номер_x'], axis = 1)

In [7]:
for i in range(len(gdf_merged)):
    if gdf_merged.loc[i, 'geometry_y'] != None:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_y'].centroid # Если геометрия не пустая, то из полигона делаем точку
    else:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_x']
gdf_merged.rename(columns={'geometry_y':'geometry'}, inplace = True)
gdf_merged = gpd.GeoDataFrame(gdf_merged,geometry='geometry', crs='EPSG:4326')
gdf_merged['geometry']=gdf_merged['geometry'].centroid

In [9]:
# готовим ДФ для последующего анализа
gdf_merged = DataMethodPreparation.start_all(gdf_merged)

# добавляекм кадастровую стоимость
cad_cost = pd.read_excel('Example_files/report_cost_1.xlsx')
cad_cost = cad_cost[['Кадастровый номер','Аварийность объекта','Экономическое устаревание', 'Эффективный возраст', 'Срок экономической жизни, лет', 'Физический износ', 'Совокупный износ','Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м', 'Кадастровая стоимость (при оценке), руб']]
gdf_merged = pd.merge(gdf_merged, cad_cost, how='left', on='Кадастровый номер')
gdf_merged = gdf_merged.drop('geometry_x', axis=1)

In [10]:
# перенести в отдельный модуль
def parcer_osm(region, tag_str, tag_def = True): #На первом месте указываем тег, на втором его значение по умолчанию выбираются все значения
    tags = {tag_str: tag_def}
    objects = ox.features.features_from_polygon(region.unary_union, tags)
    return objects

def data_osm(city):
    region = ox.geocode_to_gdf(query=city).reset_index()
    building_osm = parcer_osm(region,'building').reset_index()
    building_osm = building_osm[building_osm['element_type'] != 'node'].reset_index()
    try:
        shop = parcer_osm(region,'shop').reset_index()
    except:
        print('Не найдены магазины')
    try:
        cafe = parcer_osm(region,'amenity', ['bar', 'cafe', 'fast_food', 'food_court', 'pub', 'restaurant']).reset_index()
    except:
        print('Не найдены точки питания')
    try:
        public_transport = parcer_osm(region,'public_transport').reset_index()
    except:
        print('Не найдены остановки общественного транспорта')
    return(region, building_osm, shop, cafe, public_transport)

In [11]:
# Скачиваем данные из OSM по двум исследуемым городам
region, building_osm, shop, cafe, public_transport = data_osm('город Вологда')
#region_t, building_osm_t, shop_t, cafe_t, public_transport_t = data_osm('город Тотьма')

In [ ]:
# Создаем гексагоны для последующего анализа hex_area = 9.96 he
hex_vologda = h3fy(region, resolution=9, clip=True, buffer=False, return_geoms=True).reset_index()

In [12]:
# Выбираем объекты расположенные в городе Вологда
heritage_vologda = gpd.overlay(gdf_merged, region, how='intersection')
#heritage_totma = gpd.overlay(gdf_merged, region_t, how='intersection')

In [13]:
# Дополнительная информация из викимапии
wiki_gdf = pd.read_csv('Example_files/file_from_wiki_vologda.csv')
wiki_gdf = gpd.GeoDataFrame(wiki_gdf, geometry=gpd.points_from_xy(wiki_gdf.lon, wiki_gdf.lat), crs='EPSG:4326')

wiki_gdf = wiki_gdf.to_crs('EPSG:32637')
heritage_vologda = heritage_vologda.to_crs('EPSG:32637')

heritage_vologda['geometry'] = heritage_vologda['geometry'].buffer(20)
heritage_vologda['arch_style']=''

for i in range(len(heritage_vologda)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(heritage_vologda.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        heritage_vologda.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''

for i in range(len(building_osm)):
        # Выбор точек внутри полигона
    points_within_polygon = wiki_gdf[wiki_gdf.geometry.within(building_osm.geometry[i])]
    # Группировка значений
    grouped_values = points_within_polygon.groupby('Unnamed: 0')['arch_style'].apply(list).reset_index()
    if grouped_values.empty != True:
        building_osm.loc[i, 'arch_style'] = grouped_values['arch_style'][0][0]
    grouped_values['arch_style'] = ''




In [14]:
heritage_vologda = AspectPhysical.start_all(heritage_vologda)
heritage_vologda = AspectLaw.start_all(heritage_vologda)
heritage_vologda = AspectEconomic.start_all(heritage_vologda)

In [ ]:
heritage_vologda = AspectSpatial.start_all(heritage_vologda, shop, cafe, public_transport,building_osm)

In [21]:
heritage_vologda = General.calculate_scores(heritage_vologda)
heritage_vologda = General.risks_assastment(heritage_vologda)

In [23]:
heritage_vologda['geometry'] = heritage_vologda['geometry'].centroid

In [25]:
# Сохранение файла
heritage_vologda.to_file('heritage_vologda.geojson')